# Federico Giorgi
**fg2617**

In [65]:
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from tqdm import tqdm
import pickle
import random
import numpy as np
from collections import Counter, defaultdict
import numpy as np
from torch import FloatTensor as FT

### Instructions
For this part, fill in the required code and make the notebook work. This wll be very similar to the Skip-Gram model, but a little more difficult. Look for the """ FILL IN """ string to guide you.

In [66]:
# Where do I want to run my job. You can do "cuda" on linux machines
# DEVICE = "mps" if torch.backends.mps.is_available() else  "cpu"
DEVICE = "cuda" if torch.cuda.is_available() else  "cpu"

# The batch size in Adam or SGD
BATCH_SIZE = 512

# Number of epochs
NUM_EPOCHS = 10

# Predict from 2 words the inner word for CBOW
# I.e. I'll have a window like ["a", "b", "c"] of continuous text (each is a word)
# We'll predict each of wc = ["a", "c"] from "b" = wc for Skip-Gram
# For CBOW, we'll use ["a", "c"] to predict "b" = wo
WINDOW = 1

# Negative samples.
K = 4

In [67]:
DEVICE

'cuda'

The text8 Wikipedia corpus. 100M characters.

In [68]:
# from google.colab import drive
# drive.mount('/content/drive')

!du -h text8

f = open('text8', 'r')
text = f.read()
# One big string of size 100M
print(len(text))

96M	text8
100000000


In [69]:
punc = '!"#$%&()*+,-./:;<=>?@[\\]^_\'{|}~\t\n'

# Can do regular expressions here too
for c in punc:
    if c in text:
        text.replace(c, ' ')

In [70]:
# A very crude tokenizer you get for free: lower case and also split on spaces
# This will not work!
# Split text on space and strip each word
# You should get a list "words" which is text but each element is a word in order

# we are not using torchtext's get_tokenizer("basic_english")
# TOKENIZER = get_tokenizer("basic_english")

# -------- SOLUTION --------
def get_tokenizer(text):
    # lower case, strip edges, split on spaces
    return text.lower().strip().split()

TOKENIZER = get_tokenizer

words = TOKENIZER(text)
f = Counter(words)

print("Total tokens:", len(words))
print("Unique words:", len(f))
print("Top 5 most common:", f.most_common(5))

Total tokens: 17005207
Unique words: 253854
Top 5 most common: [('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201)]


In [71]:
# Do a very crude filter on the text which removes all very popular words
text = [word for word in words if f[word] > 5]

In [72]:
text[0:5]

['anarchism', 'originated', 'as', 'a', 'term']

In [73]:
# SOLUTION: let's create a Vocab class, in this way we can have the get_stoi() and get_itos() methods
class Vocab:
    def __init__(self, stoi, itos):
        self._stoi = stoi
        self._itos = itos
        self._vocab = set(itos)
    def get_stoi(self):
        return self._stoi
    def get_itos(self):
        return self._itos
    def get_vocab(self):
        return self._vocab
    def __len__(self):
        return len(self._itos)

def build_vocab_from_iterator(text):
    if len(text) > 0 and isinstance(text[0], list):
        text = [word for sublist in text for word in sublist]

    counts = Counter(text)
    items = [(w, c) for w, c in counts.items() ]
    items.sort(key=lambda wc: (-wc[1], wc[0]))  # decreasing frequency, then alphabetical

    itos = [w for w, _ in items]
    # itos = {i: w for i, (w, _) in enumerate(items)}
    # We use a list for itos instead of a hash map because index-to-word lookup can be done directly by position.
    # A list is simpler and faster for sequential integer keys, while a dictionary adds unnecessary overhead.

    stoi = {w: i for i, w in enumerate(itos)}

    return Vocab(stoi, itos)


VOCAB = build_vocab_from_iterator([text])
# Rebuild the vocabulary from text above
""" FILL IN """

' FILL IN '

In [74]:
# Populate these maps manually using Counter or defaltdict
# This will not work
""" FILL IN """

# word -> int hash map
stoi = VOCAB.get_stoi()
# int -> word hash map
itos = VOCAB.get_itos()

In [75]:
stoi['as']

11

In [76]:
# Total number of words; you should see about 63K as below
len(stoi)

63641

In [77]:
f = Counter(text)
# This is the probability that we pick a word in the corpus
z = {word: f[word] / len(text) for word in f}

In [78]:
threshold = 1e-5
# Probability that word is kept while subsampling
# This is explained here and sightly differet from the paper: http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/
p_keep = {word: (np.sqrt(z[word] / 0.001) + 1)*(0.0001 / z[word]) for word in f}

In [79]:
# This is in the integer space
train_dataset = [word for word in text if random.random() < p_keep[word]]

# Rebuild the vocabulary
VOCAB = build_vocab_from_iterator([train_dataset])

In [80]:
len(train_dataset)

7845607

In [81]:
# Repopulate the stoi and itos maps again now that you dropped some words
""" FILL IN """
# word -> int mapping
stoi = VOCAB.get_stoi()
# int -> word mapping
itos = VOCAB.get_itos()

In [82]:
# The vocabulary size after we do all the filters
len(VOCAB)

63641

In [83]:
# The probability we draw something for negative sampling
f = Counter(train_dataset)
p = torch.zeros(len(VOCAB))

# Downsample frequent words and upsample less frequent
s = sum([np.power(freq, 0.75) for word, freq in f.items()])

for word in f:
    p[stoi[word]] = np.power(f[word], 0.75) / s

In [84]:
# Map everything to integers
# This might not work be careful w the above ...
train_dataset = [stoi[word] for word in text]

In [85]:
# This just gets the (wc, wo) pairs that are positive - they are seen together!

# -------- SOLUTION --------
def get_tokenized_dataset(dataset, verbose=False):
    x_list = []

    for i, token in enumerate(dataset):
        m = 1  # context window size (use WINDOW if defined globally)

        # Get the left and right tokens via safe slicing
        start = max(0, i - m)
        left_tokens = dataset[start:i]

        end = min(len(dataset), i + m + 1)  # exclusive upper bound for slicing
        right_tokens = dataset[i + 1:end]

        # Only keep samples with a full context on both sides (length m each)
        if len(left_tokens) == len(right_tokens) == m:
            # Build the training row like [left_context, right_context, center]
            w_context = left_tokens + right_tokens  # length 2m
            wc = dataset[i]  # center word (integer id)

            # Extend with a single sample row (list of ints)
            x_list.extend([w_context + [wc]])
        elif verbose:
            # Optional: report skipped positions at boundaries
            print(f"Skipping i={i}: left={len(left_tokens)}, right={len(right_tokens)}")

    return x_list



In [86]:
train_x_list = get_tokenized_dataset(train_dataset, verbose=False)

In [87]:
pickle.dump(train_x_list, open('train_x_list.pkl', 'wb'))

In [88]:
train_x_list = pickle.load(open('train_x_list.pkl', 'rb'))

In [89]:
# These are (wc, wo) pairs. All are y = +1 by design
train_x_list[:10]

[[5233, 11, 3083],
 [3083, 5, 11],
 [11, 177, 5],
 [5, 1, 177],
 [177, 3133, 1],
 [1, 44, 3133],
 [3133, 59, 44],
 [44, 140, 59],
 [59, 128, 140],
 [140, 741, 128]]

In [90]:
# The number of things of BATCH_SIZE = 512
assert(len(train_x_list) // BATCH_SIZE == 32579)

### Set up the dataloader.

In [91]:
train_dl = DataLoader(
    TensorDataset(
        torch.tensor(train_x_list).to(DEVICE),
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [92]:
for xb in train_dl:
    assert(xb[0].shape == (BATCH_SIZE, 3))
    break

### Words we'll use to asses the quality of the model ...

In [93]:
valid_ids = torch.tensor([
    stoi['money'],
    stoi['lion'],
    stoi['africa'],
    stoi['musician'],
    stoi['dance'],
])

### Get the model.

In [94]:
class CBOWNegativeSampling(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CBOWNegativeSampling, self).__init__()

        # -------- SOLUTION --------
        self.A = nn.Embedding(vocab_size, embed_dim)  # context embeddings
        self.B = nn.Embedding(vocab_size, embed_dim)  # target/output embeddings
        self.init_weights()

    def init_weights(self):
        # Is this the best way? Not sure
        #initrange = 0.5
        #self.A.weight.data.uniform_(-initrange, initrange)
        #self.B.weight.data.uniform_(-initrange, initrange)

        # -------- SOLUTION --------
        initrange = 1.0 / np.sqrt(self.A.embedding_dim)
        self.A.weight.data.uniform_(-initrange, initrange)
        self.B.weight.data.uniform_(-initrange, initrange)


    def forward(self, x):
        # -------- SOLUTION --------
        # N is the batch size
        # x is (N, 3)

        # Context words are 2m things, m = 1 so w_context is (N, 2) while wc is (N, 1)
        w_context, wc = x[:, :2], x[:, 2:]          # (N, 2), (N, 1)

        # Each of these is (N, 2, D) since each context has 2 word
        # We want this to be (N, D) and this is what we get

        # (N, 2, D)
        a = self.A(w_context)

        # (N, D)
        a_avg = a.mean(dim=1)

        # Each of these is (N, D) since each target has 1 word
        b = self.B(wc).squeeze(1)

        # The product between each context and target vector. Look at the Skip-Gram code.
        # The logits is now (N, 1) since we sum across the final dimension.
        logits = torch.sum(a_avg * b, dim=1, keepdim=True)

        return logits

In [95]:
@torch.no_grad()
def validate_embeddings(
    model,
    valid_ids,
    itos
):
    """ Validation logic """

    # We will use context embeddings to get the most similar words
    # Other strategies include: using target embeddings, mean embeddings after avaraging context/target
    embedding_weights = model.A.weight

    normalized_embeddings = embedding_weights.cpu() / np.sqrt(
        np.sum(embedding_weights.cpu().numpy()**2, axis=1, keepdims=True)
    )

    # Get the embeddings corresponding to valid_term_ids
    valid_embeddings = normalized_embeddings[valid_ids, :]

    # Compute the similarity between valid_term_ids (S) and all the embeddings (V)
    # We do S x d (d x V) => S x D and sort by negative similarity
    top_k = 10 # Top k items will be displayed
    similarity = np.dot(valid_embeddings.cpu().numpy(), normalized_embeddings.cpu().numpy().T)

    # Invert similarity matrix to negative
    # Ignore the first one because that would be the same word as the probe word
    similarity_top_k = np.argsort(-similarity, axis=1)[:, 1: top_k+1]

    # Print the output.
    for i, word_id in enumerate(valid_ids):
        # j >= 1 here since we don't want to include the word itself.
        similar_word_str = ', '.join([itos[j] for j in similarity_top_k[i, :] if j >= 1])
        # This might need a fix!
        print(f"{itos[word_id]}: {similar_word_str}")

    print('\n')

### Set up the model

In [96]:
LR = 10.0
NUM_EPOCHS = 10
EMBED_DIM = 300

In [97]:
model = CBOWNegativeSampling(len(VOCAB), EMBED_DIM).to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

# The learning rate is lowered every epoch by 1/10
# Is this a good idea?
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.1)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

In [98]:
model

CBOWNegativeSampling(
  (A): Embedding(63641, 300)
  (B): Embedding(63641, 300)
)

In [99]:
validate_embeddings(model, valid_ids, itos)

money: inductor, bream, langer, acknowledgment, luo, promotion, podcasting, coelom, mishearing, macrostate
lion: disconnection, retrieving, ivf, modest, zalman, trust, reissued, triangular, ortelius, acquired
africa: arches, direct, wonderful, stem, wordplay, netted, lysis, downgraded, eradication, unaffected
musician: dunbar, businesses, exalts, struct, koalas, tulare, unsaturated, zeitschrift, azariah, hour
dance: chandigarh, mep, cfm, shellfish, ssa, brie, caymanian, cholinesterase, preponderance, ingesting




/tmp/ipython-input-1339109259.py:13: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  normalized_embeddings = embedding_weights.cpu() / np.sqrt(


### Train the model

In [100]:
ratios = []

def train(dataloader, model, optimizer, epoch):
    model.train()
    total_acc, total_count, total_loss, total_batches = 0, 0, 0.0, 0.0
    log_interval = 500

    for idx, x_batch in tqdm(enumerate(dataloader)):

        x_batch = x_batch[0]

        batch_size = x_batch.shape[0]

        # Zero the gradient so they don't accumulate
        optimizer.zero_grad()

        logits = model(x_batch)

        # Get the positive samples loss. Notice we use weights here
        positive_loss = torch.nn.functional.softplus(-logits).squeeze(1)

        # For each batch, get some negative samples
        # We need a total of len(y_batch) * K samples across a batch
        # We then reshape this batch
        # These are effectively the output words
        negative_samples = torch.multinomial(p.to(DEVICE), batch_size * K, replacement=True)  # (N*K,)

        # Context words are 2m things, m = 1 so w_context is (N, 2) while wc is (N, 1)
        w_context, wc = x_batch[:, :2], x_batch[:, 2:]

        """
        if w_context looks like below (batch_size = 3)
        [
        (a, b),
        (c, d),
        (e, f)
        ] and K = 2 we'd like to get:

        [
        (a, b),
        (a, b),
        (c, d),
        (c, d),
        (e, f),
        (e, f)
        ]

        This will be batch_size * K rows.
        """

        # This should be (N * K, 2)
        w_context = torch.concat([
            w.repeat(K, 1) for w in torch.tensor(w_context).split(1)
        ])

        # Remove the last dimension 1
        wc = wc.squeeze(1)

        # Get the negative samples. This should be (N * K, 3)
        # Concatenate the w_context and wc along the column. Make sure everything is on CUDA / MPS or CPU
        x_batch_negative = torch.cat([w_context, negative_samples.view(-1, 1)], dim=1)

        """
        Note the way we formulated the targets: they are all 0 since these are negative samples.
        We do the BCEWithLogitsLoss by hand basically here.
        Notice we sum across the negative samples, per positive word.

        This is literally the equation in the lecture notes.
        """

        # (N, K, D) -> (N, D) -> (N)
        # Look at the Skip-Gram notebook
        logits_neg = model(x_batch_negative).view(batch_size, K)
        negative_loss = torch.nn.functional.softplus(logits_neg).sum(dim=1)

        loss = (positive_loss + negative_loss).mean()

        # Get the gradients via back propagation
        loss.backward()

        # Clip the gradients? Generally a good idea
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)


        # Do an optimization step. Update the parameters A and B
        optimizer.step()

        # Get the new loss
        total_loss += loss.item()

        # Update the batch count
        total_batches += 1

        if idx % log_interval == 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| loss {:8.3f} ".format(
                    epoch,
                    idx,
                    len(dataloader),
                    total_loss / total_batches
                )
            )
            validate_embeddings(model, valid_ids, itos)
            total_loss, total_batches = 0.0, 0.0

### Some results from the run look like below:

Somewhere inside of 2 iterations you should get sensible associattions.
Paste here a screenshot of the closest vectors.

In [101]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()

    train(train_dl, model, optimizer, epoch)
    # We have a learning rate scheduler here

    # Basically, given the state of the optimizer, this lowers the learning rate in a smart way
    scheduler.step()

0it [00:00, ?it/s]/tmp/ipython-input-978102027.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w.repeat(K, 1) for w in torch.tensor(w_context).split(1)
/tmp/ipython-input-1339109259.py:13: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  normalized_embeddings = embedding_weights.cpu() / np.sqrt(
1it [00:02,  2.88s/it]

| epoch   1 |     0/32580 batches | loss    3.467 
money: inductor, bream, langer, acknowledgment, luo, promotion, podcasting, coelom, mishearing, macrostate
lion: disconnection, retrieving, ivf, modest, zalman, trust, reissued, triangular, ortelius, acquired
africa: arches, direct, wonderful, stem, wordplay, netted, lysis, downgraded, eradication, unaffected
musician: dunbar, businesses, exalts, struct, koalas, tulare, unsaturated, zeitschrift, azariah, hour
dance: chandigarh, mep, cfm, shellfish, ssa, brie, caymanian, cholinesterase, preponderance, ingesting




494it [00:09, 74.24it/s]

| epoch   1 |   500/32580 batches | loss    3.098 
money: as, more, have, new, most, was, are, but, by, be
lion: disconnection, retrieving, ivf, modest, trust, zalman, reissued, triangular, object, ortelius
africa: these, still, it, former, direct, various, king, also, wonderful, stem
musician: businesses, tulare, hour, koalas, dunbar, relator, follow, zeitschrift, struct, matheson
dance: although, chandigarh, constitution, shellfish, cfm, mep, there, are, also, ascii




995it [00:16, 73.49it/s]

...

31997it [07:56, 72.57it/s]

| epoch  10 | 32000/32580 batches | loss    0.946 
money: goods, taxes, prices, payment, buying, millions, funds, compensation, gift, cash
lion: sands, bearded, saguinus, crab, goat, swan, alder, eagle, serpent, calf
africa: korea, borneo, albania, eurasia, mozambique, yemen, americas, africans, sumatra, morocco
musician: singer, guitarist, pianist, drummer, comedian, songwriter, dancer, songwriters, bassist, singers
dance: dancing, jazz, blues, dances, ballroom, pop, bluegrass, rap, swing, techno




32501it [08:04, 44.52it/s]

| epoch  10 | 32500/32580 batches | loss    0.942 
money: goods, taxes, prices, payment, buying, millions, funds, compensation, gift, cash
lion: sands, bearded, saguinus, crab, goat, swan, alder, serpent, eagle, calf
africa: korea, borneo, albania, eurasia, mozambique, yemen, americas, africans, sumatra, morocco
musician: singer, guitarist, pianist, drummer, comedian, songwriter, dancer, songwriters, bassist, singers
dance: dancing, jazz, blues, dances, ballroom, pop, bluegrass, rap, swing, techno




32580it [08:07, 66.77it/s]


**Associated words:**

32501it [08:04, 44.52it/s]| epoch  10 | 32500/32580 batches | loss    0.942

money: goods, taxes, prices, payment, buying, millions, funds, compensation, gift, cash

lion: sands, bearded, saguinus, crab, goat, swan, alder, serpent, eagle, calf

africa: korea, borneo, albania, eurasia, mozambique, yemen, americas, africans, sumatra, morocco

musician: singer, guitarist, pianist, drummer, comedian, songwriter, dancer, songwriters, bassist, singers

dance: dancing, jazz, blues, dances, ballroom, pop, bluegrass, rap, swing, techno